-- CVRP with CPLEX:

-- volume max is included

-- sales min is included

-- max duration for each truck is included 

# Capacitated Vehicle Routing Problem

<img src="http://neo.lcc.uma.es/dynamic/images/vrp.png" width="400" />

- $n$ is the number of clientes
- $N$ is set of clients, with $N=\{1,2,...,n\}$
- $V$ is set of vetices (or nodes), with $V=\{0\}\cup N$
- $A$ is set of arcs, with $A=\{(i,j)\in V^2 : i\neq j\}$
- $c_{ij}$ is cost of travel over arc $(i,j)\in A$
- $Q$ is the vehicle capacity
- $q_i$ is the amount that has to be delivered to customer $i\in N$

Then, the formulation is the following
$$\begin{align}
\min \quad & \sum_{i,j\in A} c_{ij} x_{ij} \\
\text{s.t.} \quad & \sum_{j\in V , j\neq i} x_{ij} = 1  && i \in N \\
& \sum_{i\in V , i\neq j} x_{ij} = 1 && j \in N \\
& \text{if} \ x_{ij}=1 \ \Rightarrow \ u_i + q_j = u_j && i,j \in A : j\neq 0, i\neq 0 \\
& q_i \leq u_i \leq Q && i \in N \\
& x_{ij} \in \{0,1\} && i,j \in A
\end{align}$$

https://www.youtube.com/watch?v=-hGL39jdtQE

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 610 kB 4.2 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=dbf2c74b840cacfe82fb27dda0f3201c278713a7c0f041f52e55e031fdae8316
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


## Install and Import packages

In [3]:
!pip install geopandas
!pip install pandas_bokeh
!pip install matplotlib
!pip install plotly_express
!pip install osmnx

# !pip uninstall rtree
!pip install libspatialindex-dev
!pip install rtree
!pip install pyyaml==5.4.14

!pip install geopy
!pip install mlrose
!pip install six

!pip install ortools

!pip install pulp
!pip install gmaps
!pip install googlemaps 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.3 MB 47.0 MB/s 
     |████████████████████████████████| 16.7 MB 376 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 2.7 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 11.2 MB 34.5 MB/s 
     |████████████████████████████████| 1.0 MB 50.6 MB/s 
     |████████████████████████████████| 930 kB 51.4 MB/s 
  Attempting uninstall: requests
    Found ex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pyyaml==5.4.14 (from versions: 3.10, 3.11, 3.12, 3.13b1, 3.13rc1, 3.13, 4.2b1, 4.2b2, 4.2b4, 5.1b1, 5.1b3, 5.1b5, 5.1, 5.1.1, 5.1.2, 5.2b1, 5.2, 5.3b1, 5.3, 5.3.1, 5.4b1, 5.4b2, 5.4, 5.4.1, 6.0b1, 6.0)
ERROR: No matching distribution found for pyyaml==5.4.14
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in index

In [4]:
# you may run into this problem https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# I have manually updated site-packages/mlrose/neural.py chaning 
# original:
# from sklearn.external import six
# to:

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose # for travelling salesman problem

import pandas as pd
import numpy as np
import numpy as nph

!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt


from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

import plotly_express as px
# px.__version__
import geopandas as gpd
from shapely.geometry import Point, LineString


import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import numpy as np
import pandas as pd
import pulp
import itertools

import googlemaps
import gmaps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 5.0 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
osmnx 1.1.2 requires matplotlib>=3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [5]:
# date =[]
# num_zips =[]
# sum_vol = []

# # Get the count of non-Zeros values in columns 
# for i in df_01.columns:
# #     print(i)
#     count = (df_01[i] != 0).sum()
# #     print(count)
#     sum_ = df_01[i].sum()
# #     print(sum_)
#     sum_vol.append(sum_)
#     date.append(i)
#     num_zips.append(count)
    
#     # print(f'Count of no.n zeros in Column {i} : ', count)


# # sum_vol = sum_vol[1:]
# print(len(date), len(num_zips),len(sum_vol))

# delivery_days = pd.DataFrame({'date':date, 'num_zips':num_zips, 'sum_vol':sum_vol})
# delivery_days = delivery_days.iloc[1:,:]
# delivery_days = delivery_days.sort_values(by='num_zips')

# delivery_days[delivery_days["num_zips"] == 4]
# day = 11
# selected_days = list(delivery_days["date"][delivery_days["num_zips"] == day])
# print(selected_days)
# selected_day = selected_days[0]

In [6]:
import geopandas
from gmaps.geotraitlets import Longitude

data_path = "/content/drive/MyDrive/geo/data/"
work_path = "/content/drive/MyDrive/geo/Route_Simulation/CVRP_Romeoville/"

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


missing zipcodes are fixed, try the full code with this!

In [8]:
nan_zipcodes = []
######################### Import geo data ############################
import geopandas
from gmaps.geotraitlets import Longitude

# zips = geopandas.read_file('/content/drive/MyDrive/geo/tl_2019_us_zcta510/tl_2019_us_zcta510.shp')

# zips = pd.read_csv(f"{data_path}/Romeoville/US_geo_data.csv",encoding= 'unicode_escape')

# zips = pd.read_excel(f"{data_path}/Romeoville/Zip Data.xlsx")
# zips = zips.rename(columns = {"zip": "zipcode"})
# zips["zipcode"] = zips["zipcode"].astype(str)

data = [[46354,37.167931,-95.845016], [60057,48.890652,-116.982178], [60092,42.283056,-87.953056], [60635,41.9223,-87.8104], [60658,48.890652,-116.982178], [61009,48.890652,-116.982178]]
# Create the pandas DataFrame
missing_dipcodes_df = pd.DataFrame(data, columns = ['zipcode', 'latitude', 'longitude'])
 
# # print dataframe.
# missing_dipcodes_df

zips = pd.read_csv(f"{data_path}/Romeoville/USZIPCodeToCityStateDB.csv")
zips = zips.rename(columns = {"Zip Code": "zipcode", "ZipLatitude":"latitude", "ZipLongitude":"longitude"})
zips = zips.drop_duplicates(subset = "zipcode",keep='first')
zips = zips.dropna()
zips = zips.append(missing_dipcodes_df)
# print("####################################### GEO DATA ######################################### ")
# print(zips.head())

########################## Load Romeoville_zipcodes_By_dates_Volume ##########################
vol_df = pd.read_csv(f"{data_path}/Romeoville/Romeoville_zipcodes_By_dates_Volume.csv")
vol_df.rename(columns = {"Unnamed: 0":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE DATA with VOLUME values =====================")
# print("################################################################################")
# print(vol_df.head(2))
########################## Load Romeoville_zipcodes_By_dates_Sales ############################
sales_df = pd.read_csv(f"{data_path}/Romeoville/Romeoville_zipcodes_By_dates_Sales.csv")
sales_df.rename(columns = {"Unnamed: 0":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE DATA with SALES values =====================")
# print(sales_df.head(2))
# print("################################################################################")

######################### Load Romeoville Postal codes with Route codes (special to Romeoville: 01 or 07)  ############################
route_codes = pd.read_excel(f"{data_path}/Romeoville/Romeoville_route_codes_Postal Codes.xlsx")
route07 = route_codes[route_codes["RouteCodeID"] == "ROM07"]
route01 = route_codes[route_codes["RouteCodeID"] == "ROM01"]
route07["State"].unique(), route01["State"].unique()
# print("================= ROMEOVILLE DATA for route ROM01=====================")
# print(route01.head(2))
# print("================= ROMEOVILLE DATA for route ROM07=====================")
# print(route01.head(2))
# print("################################################################################")

######################### Load Romeoville detailed data ############################
romeoville = pd.read_csv(f"{data_path}/Romeoville/zips_954_Volume.csv")
romeoville = romeoville.drop("Unnamed: 0", axis = 1)
romeoville.rename(columns = {"postal_code":"PostalCodeID"}, inplace = True)
# print("================= ROMEOVILLE detailed DATA =====================")
# print(romeoville.head(2))
    
######################### Filter Romeoville detailed data for ROM01 and ROM07  ############################
romeoville_01 = romeoville[romeoville["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
romeoville_07 = romeoville[romeoville["PostalCodeID"].isin(route07["PostalCodeID"].unique())]

df_07_vol = vol_df[vol_df["PostalCodeID"].isin(route07["PostalCodeID"].unique())]
df_01_vol = vol_df[vol_df["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
# print("================= ROMEOVILLE ROM01 DATA (Volume) =====================")
# print(df_01_vol.head(2))
# print("################################################################################")

df_07_sales = sales_df[sales_df["PostalCodeID"].isin(route07["PostalCodeID"].unique())]
df_01_sales = sales_df[sales_df["PostalCodeID"].isin(route01["PostalCodeID"].unique())]
# print("================= ROMEOVILLE ROM01 DATA (Sales) =====================")
# print(df_01_sales.head(2))
# print("################################################################################")

zips_no_goe_data = list(set(df_01_sales["PostalCodeID"]) - set(zips["zipcode"]))
print("Number of missing zipcodes:",len(zips_no_goe_data))
print("Missing zipcodes:",zips_no_goe_data)
                


Number of missing zipcodes: 0
Missing zipcodes: []


In [9]:
df_01_vol.columns

Index(['PostalCodeID', '2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04',
       '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09',
       ...
       '2022-03-21', '2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25',
       '2022-03-26', '2022-03-27', '2022-03-28', '2022-03-29', '2022-03-30'],
      dtype='object', length=4473)

In [10]:
val_days = []
for selected_day in df_01_vol.columns[1:]:    
    # print(selected_day)
    ######################### Select the day of the orders to be delivered ############################
    # print("ORDERS (volume) DATA FOR",selected_day)
    # print(df_01_vol[selected_day])
    # print("-----------------------------------------------------------------------------------")
    ######################## Total volume of the orders to be delivered for selected day ############################
    vol_romeo_selected_01 = (df_01_vol[["PostalCodeID",selected_day]]).copy()
    vol_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"volume"}, inplace=True)
    vol_romeo_selected_01["zipcode"] = vol_romeo_selected_01["zipcode"].astype(str)
    vol_romeo_selected_01 = vol_romeo_selected_01.loc[vol_romeo_selected_01["volume"] != 0]

    # print("vol_romeo_selected_01",vol_romeo_selected_01)
    # print(type(vol_romeo_selected_01))
    # print("TOTAL VOLUME OF ORDERS FOR",selected_day)
    # print(selected_day, "\n", vol_romeo_selected_01.sum())
    # print("################################################################################")


    # print("ORDERS (sales) DATA FOR",selected_day)
    # print(df_01_sales[selected_day])
    # print("-----------------------------------------------------------------------------------")
    ######################### Total sales of the orders to be delivered for selected day ############################
    sales_romeo_selected_01 = (df_01_sales[["PostalCodeID",selected_day]]).copy()
    sales_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
    sales_romeo_selected_01["zipcode"] = sales_romeo_selected_01["zipcode"].astype(str)
    sales_romeo_selected_01 = sales_romeo_selected_01.loc[sales_romeo_selected_01["sales"] != 0]

    # print(sales_romeo_selected_01)
    # print("sales_romeo_selected_01",sales_romeo_selected_01)
    # print(type(sales_romeo_selected_01))
    # print("TOTAL SALES OF ORDERS  FOR ",selected_day)
    # print(selected_day, "\n", sales_romeo_selected_01.sum())
    # print("################################################################################")


    ######################### check if there is any order to be delivered for selected day ############################
    if vol_romeo_selected_01["volume"].sum() <= 0.0:
        # print("NO DELIVERY FOR ", selected_day)
        # # print("################################################################################")
        with open(f'{work_path}/Results/no_order_dates.txt', 'a') as k:
            k.write('\n' + selected_day)
    elif sales_romeo_selected_01["sales"].sum() <= 0.0:
        # print("Negative sales for ", selected_day)
        # print("################################################################################")
        with open(f'{work_path}/Results/negative_sales_total.txt', 'a') as n:
            n.write('\n' + selected_day)
    elif len(vol_romeo_selected_01)>5 and len(vol_romeo_selected_01)<10 :
        print(selected_day, "-NUMBER OF DELIVERY:", len(vol_romeo_selected_01))
        val_days.append(selected_day)
        print("################################################################################")

2011-08-05 -NUMBER OF DELIVERY: 6
################################################################################
2011-08-19 -NUMBER OF DELIVERY: 6
################################################################################
2014-08-21 -NUMBER OF DELIVERY: 6
################################################################################
2014-11-21 -NUMBER OF DELIVERY: 6
################################################################################
2014-12-14 -NUMBER OF DELIVERY: 8
################################################################################
2014-12-17 -NUMBER OF DELIVERY: 8
################################################################################
2014-12-19 -NUMBER OF DELIVERY: 7
################################################################################
2014-12-20 -NUMBER OF DELIVERY: 7
################################################################################
2014-12-21 -NUMBER OF DELIVERY: 6
##############################################

In [11]:
val_days, len(val_days)

(['2011-08-05',
  '2011-08-19',
  '2014-08-21',
  '2014-11-21',
  '2014-12-14',
  '2014-12-17',
  '2014-12-19',
  '2014-12-20',
  '2014-12-21',
  '2014-12-23',
  '2014-12-24',
  '2014-12-26',
  '2014-12-28',
  '2020-04-01',
  '2020-04-06',
  '2020-05-24',
  '2020-06-08',
  '2020-12-06'],
 18)

In [12]:
######################### Select the day of the orders to be delivered ############################
selected_day = "2020-04-06" # df_01_vol.columns[i+1] 2020-12-06
print("-- SELECTED DAY is ", selected_day)

print("ORDERS (volume) DATA FOR",selected_day)
print(df_01_vol[selected_day])
print("-----------------------------------------------------------------------------------")
######################## Total volume of the orders to be delivered for selected day ############################
vol_romeo_selected_01 = (df_01_vol[["PostalCodeID",selected_day]]).copy()
vol_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"volume"}, inplace=True)
vol_romeo_selected_01["zipcode"] = vol_romeo_selected_01["zipcode"].astype(str)
vol_romeo_selected_01 = vol_romeo_selected_01.loc[vol_romeo_selected_01["volume"] != 0]

# print("vol_romeo_selected_01",vol_romeo_selected_01)
# print(type(vol_romeo_selected_01))
print("TOTAL VOLUME OF ORDERS FOR",selected_day)
print(selected_day, "\n", vol_romeo_selected_01.sum())
print("################################################################################")


print("ORDERS (sales) DATA FOR",selected_day)
print(df_01_sales[selected_day])
print("-----------------------------------------------------------------------------------")
######################### Total sales of the orders to be delivered for selected day ############################
sales_romeo_selected_01 = (df_01_sales[["PostalCodeID",selected_day]]).copy()
sales_romeo_selected_01.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
sales_romeo_selected_01["zipcode"] = sales_romeo_selected_01["zipcode"].astype(str)
sales_romeo_selected_01 = sales_romeo_selected_01.loc[sales_romeo_selected_01["sales"] != 0]

# print(sales_romeo_selected_01)
# print("sales_romeo_selected_01",sales_romeo_selected_01)
# print(type(sales_romeo_selected_01))
print("TOTAL SALES OF ORDERS  FOR ",selected_day)
print(selected_day, "\n", sales_romeo_selected_01.sum())
print("################################################################################")


######################### check if there is any order to be delivered for selected day ############################
if vol_romeo_selected_01["volume"].sum() <= 0.0:
    print("NO DELIVERY FOR ", selected_day)
    # print("################################################################################")
    with open(f'{work_path}/Results/no_order_dates.txt', 'a') as k:
        k.write('\n' + selected_day)
elif sales_romeo_selected_01["sales"].sum() <= 0.0:
    print("Negative sales for ", selected_day)
    print("################################################################################")
    with open(f'{work_path}/Results/negative_sales_total.txt', 'a') as n:
        n.write('\n' + selected_day)
else:
    print("NUMBER OF DELIVERY:", len(vol_romeo_selected_01))
    print("################################################################################")
    
    # ######################### check if there is any order to be delivered for selected day ############################
    # sales_romeo_selected = (sales_df[["PostalCodeID",selected_day]]).copy()
    # sales_romeo_selected.rename(columns = {"PostalCodeID":"zipcode",selected_day:"sales"}, inplace=True)
    # sales_romeo_selected["zipcode"] = sales_romeo_selected["zipcode"].astype(str)
    # sales_romeo_selected = sales_romeo_selected.loc[sales_romeo_selected["sales"] != 0]
    # print(sales_romeo_selected)
    # print("################################################################################")

    sales_vol_romeo_selected_01 = pd.merge(sales_romeo_selected_01, vol_romeo_selected_01,  how="right", on=["zipcode"])
    sales_vol_romeo_selected_01["zipcode"] = sales_vol_romeo_selected_01["zipcode"].astype(int)
    sales_vol_romeo_selected_01[["sales", "volume"]] = sales_vol_romeo_selected_01[["sales", "volume"]].fillna(0)

    # print("TYPE sales_vol_romeo_selected_01",type(sales_vol_romeo_selected_01["zipcode"][0]))
    # print("TYPE zips", type(zips["zipcode"][0]))
    print("sales_vol_romeo_selected_01", sales_vol_romeo_selected_01)
    # print(type(sales_vol_romeo_selected_01["zipcode"][0]))
    # print("SALES and VOLUME DATA FOR",selected_day)
    # print(sales_vol_romeo_selected_01)
    # print("TOTAL SALES and VOLUME OF ORDERS  FOR",selected_day)
    # print(sales_vol_romeo_selected_01.sum())
    # print("################################################################################")

    # print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    print("####################################### ZIP CODE OF THE DISTRIBUTION CENTER ######################################### ")
    # depot_zip = input("Enter DC zipcode: ")
    depot = zips[zips['zipcode'] == 60446] #  f"{depot_zip}" 60446 is Romeoville DC zipcode
    depot = depot[["zipcode", "latitude","longitude"]]
    print("DATA FOR of DC:")
    print(depot)


    df = pd.merge(sales_vol_romeo_selected_01, zips,  how="left", on=["zipcode"])
    print("sales_vol_romeo_selected_01",sales_vol_romeo_selected_01)
    df = df.append(depot)
    df = df[["zipcode", "latitude",	"longitude", "sales", "volume"]]
    # df = df.rename(columns = {"zipcode":"zipcode",	"lat":"latitude","lng":"longitude"})
    df = df.reset_index(drop=True)
    print("First version of dataframe")
    print(df)

    ###################### remove the plus(+) sign from the latitude ######################
    # for j in range(len(df)):
    #     df['latitude'].iloc[j] = (df.latitude[j]).strip("+")

    ###################### Add DC as the starting point as the first row data and initialize it to 0 ######################
    df['new_col'] = range(1, len(df)+1)
    df['new_col'].loc[len(df)-1] = 0
    df['new_col'].loc[0] = len(df)
    df = df.sort_values(by = "new_col")
    df = df.reset_index(drop=True)
    df.drop("new_col", axis = 1, inplace=True)
    df.iloc[0, 3] = 0
    df.iloc[0, 4] = 0
    print("################# ZipCodes data with latitude and longitude #################")
    print(df)

    ###################### Create sales and volume data with latitude and longitude ######################          
    df_vol = df[["latitude","longitude","volume"]]
    # print("################# VOLUME DATAFRAME #################")
    df_vol["latitude"], df_vol["longitude"] = df_vol["latitude"].astype(str), df_vol["longitude"].astype(str) 
    print(df_vol)

    df_sales = df[["latitude","longitude","sales"]]
    # print("################# SALES DATAFRAME #################")
    print(df_sales)

    for j in range(len(df)):
        if  math.isnan(df["latitude"][j])  and df["zipcode"][j] != 60446:
            print("zipcode:", df["zipcode"][j])
            nan_zipcodes.append(df["zipcode"][j])

        # if df["zipcode"][j] in ["46354", "60057", "60092", "60635", "60658", "61009"]:
        #     print("DIKKKAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT")


-- SELECTED DAY is  2020-04-06
ORDERS (volume) DATA FOR 2020-04-06
106    0.0
107    0.0
108    0.0
109    0.0
110    0.0
      ... 
963    0.0
964    0.0
965    0.0
966    0.0
967    0.0
Name: 2020-04-06, Length: 732, dtype: float64
-----------------------------------------------------------------------------------
TOTAL VOLUME OF ORDERS FOR 2020-04-06
2020-04-06 
 zipcode    4632346373600046001360102601486050460565
volume                                        28.48
dtype: object
################################################################################
ORDERS (sales) DATA FOR 2020-04-06
106    0.0
107    0.0
108    0.0
109    0.0
110    0.0
      ... 
963    0.0
964    0.0
965    0.0
966    0.0
967    0.0
Name: 2020-04-06, Length: 732, dtype: float64
-----------------------------------------------------------------------------------
TOTAL SALES OF ORDERS  FOR  2020-04-06
2020-04-06 
 zipcode    4632346373600046001360102601486050460565
sales                                     

In [13]:
df = df.reset_index()

In [14]:
df

,index,zipcode,latitude,longitude,sales,volume
0,0,60446,41.636022,-88.095961,0.00,0.00
1,1,46373,41.450011,-87.472237,135.20,2.94
2,2,60004,42.112929,-87.981178,95.99,2.46
3,3,60013,42.225542,-88.231746,356.00,3.46
4,4,60102,42.158070,-88.312323,133.50,3.46
5,5,60148,41.877628,-88.019802,119.20,2.57
6,6,60504,41.748954,-88.245568,79.20,5.30
7,7,60565,41.726899,-88.112509,239.20,4.83
8,8,46323,41.588430,-87.456695,133.50,3.46


In [15]:
# customer count ('0' is depot) 
customer_count = len(df)-1

# the number of vehicle
vehicle_count = 5

# the capacity of vehicle
vehicle_capacity = 10

# fix random seed
np.random.seed(seed=777)

# set depot latitude and longitude
depot_latitude = df.iloc[0,2]
depot_longitude = df.iloc[0,3] 

In [16]:
bgbng

NameError: ignored

In [17]:
df

,index,zipcode,latitude,longitude,sales,volume
0,0,60446,41.636022,-88.095961,0.00,0.00
1,1,46373,41.450011,-87.472237,135.20,2.94
2,2,60004,42.112929,-87.981178,95.99,2.46
3,3,60013,42.225542,-88.231746,356.00,3.46
4,4,60102,42.158070,-88.312323,133.50,3.46
5,5,60148,41.877628,-88.019802,119.20,2.57
6,6,60504,41.748954,-88.245568,79.20,5.30
7,7,60565,41.726899,-88.112509,239.20,4.83
8,8,46323,41.588430,-87.456695,133.50,3.46


In [ ]:
df_vol = df[["latitude", "longitude", "volume"]]

In [ ]:
df_vol

In [ ]:
#Problem: _distance_calculator function does not work without GCP API

In [ ]:
_distance_result = np.zeros((len(df),len(df)))
_distance_result

In [ ]:
# # function for plotting on google maps
# def _plot_on_gmaps(_df):
    
#     _marker_locations = []
#     for i in range(len(_df)):
#         _marker_locations.append((_df['latitude'].iloc[i],_df['longitude'].iloc[i]))
    
#     _fig = gmaps.figure()
#     _markers = gmaps.marker_layer(_marker_locations)
#     _fig.add_layer(_markers)

#     return _fig

# # function for calculating distance between two pins
# def _distance_calculator(_df):
    
#     _distance_result = np.zeros((len(_df),len(_df)))
#     _df['latitude-longitude'] = '0'
#     for i in range(len(_df)):
#         _df['latitude-longitude'].iloc[i] = str(_df.latitude[i]) + ',' + str(_df.longitude[i])
    
#     for i in range(len(_df)):
#         for j in range(len(_df)):
            
#             # calculate distance of all pairs
#             _google_maps_api_result = googlemaps.directions(_df['latitude-longitude'].iloc[i],
#                                                             _df['latitude-longitude'].iloc[j],
#                                                             mode = 'driving')
#             # append distance to result list
#             _distance_result[i][j] = _google_maps_api_result[0]['legs'][0]['distance']['value']
    
#     return _distance_result

# distance = _distance_calculator(df)
# plot_result = _plot_on_gmaps(df)
# plot_result

## costs : distance and/or time

In [ ]:
df

In [ ]:
def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    ''' durations array of arrays that stores the matrix in row-major order. durations[i][j] gives the travel time from the i-th waypoint to the j-th waypoint. 
    Values are given in seconds. Can be null if no route between i and j can be found. '''

    '''distance: The distance, in metres, between each pair of coordinates '''
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["longitude"]},{point1["latitude"]};{point2["longitude"]},{point2["latitude"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
ce_cities = df[["zipcode",'latitude','longitude']]
# get the distance and duration from OSRM API for all our combinations.
ce_cities

In [ ]:

dist_array = []
dist_cost = []
dur_cost = []
for i , r in ce_cities.iterrows():
    point1 = {"latitude": r["latitude"], "longitude": r["longitude"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"latitude": o["latitude"], "longitude": o["longitude"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array.append((df.iloc[i,0], df.iloc[j,0], duration*0.0166667, dist*0.000621371)) #df.iloc[i,0], df.iloc[j,0]
        dist_cost.append((df.iloc[i,0], df.iloc[j,0], dist*0.000621371)) #df.iloc[i,0], df.iloc[j,0]
        dur_cost.append((df.iloc[i,0], df.iloc[j,0], duration*0.0166667)) #df.iloc[i,0], df.iloc[j,0]


dist_cost

In [ ]:
# distance
c_dist= {}
for l2 in dist_array:
    x = c_dist[l2[:2]] = l2[3]

c_dist

In [ ]:
distance = pd.DataFrame(columns = range(0,len(df_vol)), index = range(0,len(df_vol)))
dist_df = pd.DataFrame(dist_cost)
dist_df = dist_df.reset_index()
dist_df

In [ ]:
distance

In [ ]:
for i in range(len(df_vol)):
    for j in range(len(df_vol)):
        if i==j:
            distance.iloc[i,j] = 0
        else:
            # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
            a = dist_df["index"].loc[(dist_df[0] == i) & (dist_df[1] == j)]
            print(a)
            distance.iloc[i,j] = float(dist_df.iloc[a,3].values)

In [ ]:
# duration
c_dur = {}
for l2 in dist_array:
    x = c_dur[l2[:2]] = l2[2]

c_dur

In [ ]:
duration = pd.DataFrame(columns = range(0,len(df_vol)), index = range(0,len(df_vol)))
dur_df = pd.DataFrame(dur_cost)
dur_df = dur_df.reset_index()
dur_df

In [ ]:
df_vol

In [ ]:
for i in range(len(df_vol)):
    for j in range(len(df_vol)):
        if i==j:
            duration.iloc[i,j] = 0
        else:
            # print("i:",i,"j:",j, "index:", x_df["index"].loc[(x_df[0] == i) & (x_df[1] == j)])
            a = dur_df["index"].loc[(dur_df[0] == i) & (dur_df[1] == j)]
            duration.iloc[i,j] = float(dur_df.iloc[a,3].values)

In [ ]:
distance

In [ ]:
df

In [ ]:
retail_capacity = 1
vehicle_capacity = 10 #df.volume.sum() + 1
vehicle_count = 5

In [ ]:
# solve with pulp
for vehicle_count in range(1,vehicle_count+1):
    
    # definition of LpProblem instance
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    # definition of variables which are 0/1
    x = [[[pulp.LpVariable("x%s_%s,%s"%(i,j,k), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)]

    # add objective function
    problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0
                          for k in range(vehicle_count) 
                          for j in range(customer_count) 
                          for i in range (customer_count))

    # constraints
    # foluma (2)
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                              for i in range(customer_count) 
                              for k in range(vehicle_count)) == 1 

    # foluma (3)
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1,customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1,customer_count)) == 1

    # foluma (4)
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                  for i in range(customer_count)) -  pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0

    #foluma (5)
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.volume[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) <= vehicle_capacity 

    #foluma (5) retail value
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.sales[j] * x[i][j][k] if i != j else 0 for i in range(customer_count) for j in range (1,customer_count)) >= retail_capacity 
   
   # fomula (6)
    subtours = []
    for i in range(2,customer_count):
         subtours += itertools.combinations(range(1,customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i !=j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s) - 1

    
    # print vehicle_count which needed for solving problem
    # print calculated minimum distance value
    if problem.solve() != 1:
        print(f"for {vehicle_count} vehicles. there is no optimal solution!")

    else:
        print('Vehicle Requirements:', vehicle_count)
        
        print('Moving Distance:', pulp.value(problem.objective))

        for v in problem.variables():
            if v.varValue>0:
                print(v.name, "=", v.varValue)
        break
        

            

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# output.enable_custom_widget_manager()

In [ ]:
# visualization : plotting on google maps
fig = gmaps.figure()
layer = []
color_list = ["red","blue","green"]

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                layer.append(gmaps.directions.Directions(
                    (df.latitude[i],df.longitude[i]),
                    (df.latitude[j],df.longitude[j]),
                    mode='car',stroke_color=color_list[k],stroke_opacity=1.0, stroke_weight=5.0))

In [ ]:
# visualization : plotting with matplolib
plt.figure(figsize=(8,8))
for i in range(customer_count):    
    if i == 0:
        plt.scatter(df.latitude[i], df.longitude[i], c='green', s=200)
        plt.text(df.latitude[i], df.longitude[i], "depot", fontsize=12)
    else:
        plt.scatter(df.latitude[i], df.longitude[i], c='orange', s=200)
        plt.text(df.latitude[i], df.longitude[i], str(df.volume[i]), fontsize=12)

for k in range(vehicle_count):
    for i in range(customer_count):
        for j in range(customer_count):
            if i != j and pulp.value(x[i][j][k]) == 1:
                plt.plot([df.latitude[i], df.latitude[j]], [df.longitude[i], df.longitude[j]], c="black")

plt.show()

In [ ]:
df

In [ ]:
for i in range(len(layer)):
    fig.add_layer(layer[i])
    
fig

## ###################################